# LangChain Built-in Tool: DuckDuckGo (Web Search)

## Introduction

`DuckDuckGoSearchRun` is a powerful tool within the LangChain framework that integrates DuckDuckGo's search functionality into your applications. Designed for developers and AI practitioners, this tool allows you to perform web searches programmatically, retrieve real-time information, and integrate search results into your workflows. Whether you're building chatbots, research assistants, or data pipelines, `DuckDuckGoSearchRun` provides a seamless way to access up-to-date information from the web without compromising user privacy, as DuckDuckGo is known for its privacy-first approach.

The tool is highly customizable, supporting features like retries, fallbacks, and configurable alternatives, making it robust and adaptable to various use cases. With its simple API and integration into the LangChain ecosystem, `DuckDuckGoSearchRun` is an essential component for applications that require dynamic, real-time data retrieval.

---

## Preparation

### Installing Required Libraries
This section installs the necessary Python libraries for working with LangChain, OpenAI embeddings, Anthropic models, DuckDuckGo search, and other utilities. These libraries include:
- `langchain-openai`: Provides integration with OpenAI's embedding models and APIs.
- `langchain-anthropic`: Enables integration with Anthropic's models and APIs.
- `langchain_community`: Contains community-contributed modules and tools for LangChain.
- `langchain_experimental`: Includes experimental features and utilities for LangChain.
- `langgraph`: A library for building and visualizing graph-based workflows in LangChain.
- `duckduckgo-search`: Enables programmatic access to DuckDuckGo's search functionality.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langgraph
!pip install -qU duckduckgo-search

### Initializing OpenAI and Anthropic Chat Models
This section demonstrates how to securely fetch API keys for OpenAI and Anthropic using Kaggle's `UserSecretsClient` and initialize their respective chat models. The `ChatOpenAI` and `ChatAnthropic` classes are used to create instances of these models, which can be used for natural language processing tasks such as text generation and conversational AI.

**Key steps:**
1. **Fetch API Keys**: The OpenAI and Anthropic API keys are securely retrieved using Kaggle's `UserSecretsClient`.
2. **Initialize Chat Models**:
   - The `ChatOpenAI` class is initialized with the `gpt-4o-mini` model and the fetched OpenAI API key.
   - The `ChatAnthropic` class is initialized with the `claude-3-5-latest` model and the fetched Anthropic API key.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from kaggle_secrets import UserSecretsClient

# Fetch API key securely
user_secrets = UserSecretsClient()

# Initialize LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=user_secrets.get_secret("my-openai-api-key"))
#model = ChatAnthropic(model="claude-3-5-latest", temperature=0, api_key=user_secrets.get_secret("my-anthropic-api-key"))

---

## 1. Initialization and Setup

### Example 1: Basic Initialization
This example demonstrates how to initialize the `DuckDuckGoSearchRun` tool with default settings.**

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the DuckDuckGoSearchRun tool
tool = DuckDuckGoSearchRun()

# Example usage
result = tool.invoke("Python programming")

word_count = len(result.split())
print("Number of words:", word_count)
print(result)

### Example 2: Custom Initialization with Configuration
This example shows how to initialize the tool with custom configurations, such as setting a description and enabling verbose logging.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool with custom settings
tool = DuckDuckGoSearchRun(
    description="A custom DuckDuckGo search tool for finding programming resources.",
    verbose=True
)

# Example usage
result = tool.invoke("LangChain framework")

word_count = len(result.split())
print("\nNumber of words:", word_count)
print(result)

---

## 2. Search Execution

### Example 1: Simple Search Execution
This example demonstrates how to perform a simple search using the `invoke` method.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool
tool = DuckDuckGoSearchRun()

# Perform a search
result = tool.invoke("latest AI news")

word_count = len(result.split())
print("\nNumber of words:", word_count)
print(result)

### Example 2: Search with ToolCall Input
This example shows how to use the tool with a `ToolCall` input, which is useful for structured inputs.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool
tool = DuckDuckGoSearchRun()

# Perform a search using ToolCall input
tool_input = {
    "args": {"query": "machine learning trends 2023"},
    "id": "1",
    "name": tool.name,
    "type": "tool_call"
}
result = tool.invoke(tool_input)

# Access specific properties (if applicable)
if hasattr(result, 'tool_call_id'):
    print("Tool Call ID:", result.tool_call_id)
if hasattr(result, 'name'):
    print("Name:", result.name)
if hasattr(result, 'content'):
    print("Content:", result.content)

---

## 3. Streaming and Batch Processing

### Example 1: Batch Processing
This example demonstrates how to process multiple search queries in batch mode.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool
tool = DuckDuckGoSearchRun()

# List of queries to process
queries = ["Python programming", "machine learning", "data science"]

# Perform batch processing
results = tool.batch(queries)
for result in results:
    print(result)
    print("-"*80)

### Example 2: Streaming Search Results
This example shows how to stream search results for a single query.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool
tool = DuckDuckGoSearchRun()

# Stream search results
query = "latest AI news"
for result in tool.stream(query):
    print(result)

---

## 4. Error Handling and Retries

### Retry Configuration Parameters

1. **`retry_if_exception_type`**:
   - This parameter specifies the types of exceptions that should trigger a retry.
   - In your example, `retry_if_exception_type=(Exception,)` means that **any exception** (since `Exception` is the base class for all exceptions in Python) will trigger a retry.

2. **`stop_after_attempt`**:
   - This parameter specifies the maximum number of retry attempts before giving up.
   - In your example, `stop_after_attempt=3` means the tool will retry up to **3 times** (including the initial attempt) before raising the exception if it continues to fail.

3. **`wait_exponential_jitter`** (not used in your example but worth mentioning):
   - If enabled, this adds a random "jitter" (small delay) to the wait time between retries, which helps avoid thundering herd problems in distributed systems.
   - The wait time between retries grows exponentially (e.g., 1s, 2s, 4s, etc.) with jitter added to avoid synchronized retries.

### Example 1: Retry on Failure
This example demonstrates how to configure the tool to retry on specific exceptions.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool with retry configuration
tool = DuckDuckGoSearchRun().with_retry(
    retry_if_exception_type=(Exception,),  # Retry on any exception
    stop_after_attempt=3                   # Retry up to 3 times
)

# Perform a search with retry logic
result = tool.invoke("Python programming")
print(result)

In [ ]:
# Exponential Backoff with Jitter
tool = DuckDuckGoSearchRun().with_retry(
    retry_if_exception_type=(Exception,),
    wait_exponential_jitter=True,  # Enable exponential backoff with jitter
    stop_after_attempt=3
)

# Perform a search with retry logic
result = tool.invoke("Python programming")
print(result)

In [ ]:
# Specific Exception Types:
# If you only want to retry on specific exceptions (e.g., ConnectionError or TimeoutError), you can specify them.
tool = DuckDuckGoSearchRun().with_retry(
    retry_if_exception_type=(ConnectionError, TimeoutError),
    stop_after_attempt=3
)

# Perform a search with retry logic
result = tool.invoke("Python programming")
print(result)

### 5. How Fallbacks Work

1. **Primary Tool**:
   - The primary tool is the first tool in the sequence. In your example, this is the `DuckDuckGoSearchRun` instance (`primary_tool`).
   - When you invoke the tool (e.g., `tool.invoke("Python programming")`), the primary tool is executed first.

2. **Fallback Tools**:
   - If the primary tool fails (e.g., raises an exception), the fallback mechanism kicks in.
   - The fallback tools are tried in the order they are specified in the `with_fallbacks` method. In your example, there is one fallback tool (`fallback_tool`), which is another instance of `DuckDuckGoSearchRun`.

3. **Execution Flow**:
   - The primary tool is executed first.
   - If the primary tool succeeds, its result is returned, and the fallback tools are not used.
   - If the primary tool fails, the first fallback tool is executed.
   - If the first fallback tool fails, the next fallback tool is executed, and so on, until either:
     - A tool succeeds and returns a result.
     - All tools fail, and an exception is raised.

4. **Exception Handling**:
   - By default, if all tools fail, the exception from the last tool is raised.
   - You can customize how exceptions are handled using the `exceptions_to_handle` and `exception_key` parameters in the `with_fallbacks` method (not shown in your example).

### Example 2: Fallback on Failure
This example shows how to configure fallback behavior if the primary search fails.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the primary tool
primary_tool = DuckDuckGoSearchRun()

# Initialize a fallback tool (e.g., another search tool or a mock response)
fallback_tool = DuckDuckGoSearchRun(description="Fallback search tool")

# Configure the tool with fallbacks
tool = primary_tool.with_fallbacks([fallback_tool])

# Perform a search with fallback logic
result = tool.invoke("Python programming")
print(result)

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the primary tool
primary_tool = DuckDuckGoSearchRun()

# Initialize fallback tools
fallback_tool_1 = DuckDuckGoSearchRun(description="Fallback tool 1")
fallback_tool_2 = DuckDuckGoSearchRun(description="Fallback tool 2")

# Configure the tool with multiple fallbacks
tool = primary_tool.with_fallbacks([fallback_tool_1, fallback_tool_2])

# Perform a search with fallback logic
result = tool.invoke("Python programming")
print(result)

---

## 6. Configuration and Binding

### Example 1: Binding Additional Arguments

#### Description
**Binding** allows you to attach additional arguments or configurations to a tool, creating a new instance of the tool with those arguments pre-set. This is useful when you want to reuse a tool with specific settings without having to pass those settings every time you invoke the tool.

In this example:
- The `DuckDuckGoSearchRun` tool is initialized.
- The `bind` method is used to attach a custom argument (`query_filter="site:github.com"`) to the tool.
- The bound tool (`custom_tool`) is then invoked with a search query, and the custom argument is automatically applied.

#### How It Works
1. **Initialization**:
   - The `DuckDuckGoSearchRun` tool is created with default settings.

2. **Binding**:
   - The `bind` method is called on the tool, passing the argument `query_filter="site:github.com"`.
   - This creates a new instance of the tool (`custom_tool`) with the `query_filter` argument pre-set.

3. **Invocation**:
   - When `custom_tool.invoke("Python programming")` is called, the search query `"Python programming"` is combined with the bound argument `query_filter="site:github.com"`.
   - The tool performs the search, filtering results to only include those from `github.com`.

4. **Output**:
   - The search results are returned and printed.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool
tool = DuckDuckGoSearchRun()

# Bind additional arguments (e.g., a custom search filter)
custom_tool = tool.bind(query_filter="site:github.com")

# Perform a search with the bound arguments
result = custom_tool.invoke("Python programming")
print(result)

### Example 2: Configurable Alternatives

#### Description
**Configurable alternatives** allow you to define multiple versions of a tool and switch between them at runtime. This is useful when you want to provide different implementations or behaviors for the same tool, depending on the context or configuration.

In this example:
- A primary tool (`primary_tool`) and an alternative tool (`alternative_tool`) are initialized.
- The `configurable_alternatives` method is used to configure the tool with these alternatives.
- The tool can then be invoked using either the primary or alternative implementation, depending on the runtime configuration.

#### How It Works
1. **Initialization**:
   - The primary tool (`primary_tool`) and an alternative tool (`alternative_tool`) are created.

2. **Configuration**:
   - The `configurable_alternatives` method is called on the primary tool, passing:
     - A `ConfigurableField` instance with an `id` of `"search_tool"`.
     - A `default_key` of `"primary"`, which specifies the default tool to use.
     - An `alternative` tool (`alternative_tool`), which can be selected at runtime.

3. **Default Invocation**:
   - When `tool.invoke("Python programming")` is called, the primary tool is used by default.

4. **Alternative Invocation**:
   - When `tool.with_config(configurable={"search_tool": "alternative"}).invoke("Python programming")` is called, the alternative tool is used instead.

5. **Output**:
   - The search results from the selected tool are returned and printed.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.runnables.utils import ConfigurableField

# Initialize the primary tool
primary_tool = DuckDuckGoSearchRun()

# Initialize an alternative tool (e.g., a mock search tool)
alternative_tool = DuckDuckGoSearchRun(description="Alternative search tool")

# Configure the tool with alternatives
tool = primary_tool.configurable_alternatives(
    ConfigurableField(id="search_tool"),
    default_key="primary",
    alternative=alternative_tool
)

# Perform a search using the default tool
result = tool.invoke("Python programming")
print(result)

# Perform a search using the alternative tool
result = tool.with_config(configurable={"search_tool": "alternative"}).invoke("Python programming")
print(result)

---

## Best Practices

### Example 1: Enhancing a Chatbot with Real-Time Web Search
This example demonstrates how to integrate `DuckDuckGoSearchRun` into a chatbot powered by a large language model (LLM) to provide real-time, up-to-date information. The chatbot can answer questions about current events, recent news, or any topic that requires live data from the web.

#### Explanation
1. **Search Tool Integration**:
   - The `DuckDuckGoSearchRun` tool is used to perform real-time web searches.
   - The search results are passed to the LLM as part of the input.

2. **LLM Prompting**:
   - The LLM is instructed to use the search results to provide accurate and up-to-date answers.

3. **Output**:
   - The chatbot combines the LLM's reasoning capabilities with real-time search results to answer user queries effectively.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the DuckDuckGoSearchRun tool
search_tool = DuckDuckGoSearchRun()

# Define a prompt template for the chatbot
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the search tool to find up-to-date information if needed."),
    ("user", "{input}")
])

# Create a chain that integrates the search tool and LLM
chain = (
    {"input": lambda x: x["input"], "search_results": lambda x: search_tool.invoke(x["input"])}
    | prompt
    | model
    | StrOutputParser()
)

# Example usage
response = chain.invoke({"input": "What are the latest trends in AI for 2023?"})
print(response)

### Example 2: Building a Research Assistant
This example shows how to create a research assistant that uses `DuckDuckGoSearchRun` to gather information on a specific topic and summarize it using an LLM. This is particularly useful for tasks like market research, academic studies, or competitive analysis.

#### Explanation
1. **Search Tool Integration**:
   - The `DuckDuckGoSearchRun` tool is used to gather information on a specific topic (e.g., renewable energy trends in 2023).

2. **LLM Summarization**:
   - The search results are passed to the LLM, which summarizes the information into a concise and readable format.

3. **Output**:
   - The research assistant provides a well-structured summary of the latest information on the topic, making it easier for users to understand key insights.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the DuckDuckGoSearchRun tool
search_tool = DuckDuckGoSearchRun()

# Define a prompt template for summarization
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research assistant. Summarize the following information:"),
    ("user", "{search_results}")
])

# Create a chain that integrates the search tool and LLM
chain = (
    {"search_results": lambda x: search_tool.invoke(x["topic"])}
    | prompt
    | model
    | StrOutputParser()
)

# Example usage
topic = "renewable energy trends in 2023"
summary = chain.invoke({"topic": topic})
print(f"Summary for '{topic}':\n{summary}")

---

## Conclusion

`DuckDuckGoSearchRun` is a versatile and reliable tool for integrating web search capabilities into your applications. Its privacy-focused design, combined with LangChain's powerful features like retries, fallbacks, and configurable alternatives, makes it an excellent choice for developers building intelligent systems that require real-time information. Whether you're creating a chatbot, automating research tasks, or enhancing data pipelines, `DuckDuckGoSearchRun` provides the flexibility and robustness needed to handle dynamic search requirements.

By leveraging this tool, you can ensure your applications stay informed with the latest data while maintaining a commitment to user privacy. As the demand for real-time, accurate information grows, `DuckDuckGoSearchRun` stands out as a key enabler for innovative and privacy-conscious solutions.